# Поиск выбросов
Здеся мы будем находить выбросы с помощью разлчны критериев.
В качестве таблицы с данными будут использоваться девушки из Плейбоя

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('girls.csv', sep=';')
df.head(20)

,Month,Year,G,T,B,L,W
0,December,1953,91,61,91,165,54
1,March,1954,86,61,89,168,54
2,July,1955,91,61,91,166,52
3,December,1955,91,61,91,166,52
4,July,1956,91,61,86,157,48
5,August,1956,94,61,91,170,55
6,October,1956,91,61,91,166,52
7,November,1956,99,56,91,160,58
8,December,1956,89,58,89,157,48
9,January,1957,91,64,91,165,54


In [2]:
df.describe()

,Year,G,T,B,L,W
count,604.000000,604.000000,604.000000,604.000000,604.000000,604.000000
mean,1983.057947,89.293046,59.529801,87.942053,167.887417,52.168874
std,14.843740,3.994011,3.616909,3.479142,5.776711,4.040585
min,1953.000000,81.000000,46.000000,61.000000,150.000000,42.000000
25%,1970.000000,86.000000,58.000000,86.000000,165.000000,49.000000
50%,1983.000000,89.000000,61.000000,89.000000,168.000000,52.000000
75%,1996.000000,91.000000,61.000000,91.000000,173.000000,54.000000
max,2009.000000,104.000000,89.000000,99.000000,188.000000,68.000000


In [3]:
df.median()# хотя это считалось выше как 50%квартиль

Year    1983.0
G         89.0
T         61.0
B         89.0
L        168.0
W         52.0
dtype: float64

In [4]:
df.corr()

,Year,G,T,B,L,W
Year,1.000000,-0.461384,0.314772,-0.310939,0.306172,-0.005168
G,-0.461384,1.000000,0.062017,0.455448,0.090848,0.408590
T,0.314772,0.062017,1.000000,0.287631,0.374636,0.468535
B,-0.310939,0.455448,0.287631,1.000000,0.231788,0.549841
L,0.306172,0.090848,0.374636,0.231788,1.000000,0.709458
W,-0.005168,0.408590,0.468535,0.549841,0.709458,1.000000


А теперь в эту таблицу я добавлю себя )))))
Интересно, буду ли я выбросом относительно других объектов?

In [5]:
df.tail(1)#это последняя запись в таблице

,Month,Year,G,T,B,L,W
603,January,2009,86,60,91,170,55


In [6]:
I_am=pd.DataFrame({'Month':'February','Year':2009,'G':100,'T':100,'B':100,'L':190,'W':100},index=[604])
frames=[df,I_am]
df_new=pd.concat(frames)
df_new.tail(2)#вывели на экран. Действительно, добавили

,B,G,L,Month,T,W,Year
603,91,86,170,January,60,55,2009
604,100,100,190,February,100,100,2009


In [7]:
#Посмотрим вначале, как изменились характеристики признаков
df_new.describe()# они увеличились не сильно, так как мои параметры распределились между 604 девушками

,B,G,L,T,W,Year
count,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000
mean,87.961983,89.310744,167.923967,59.596694,52.247934,1983.100826
std,3.510657,4.014374,5.841519,3.970836,4.481162,14.868900
min,61.000000,81.000000,150.000000,46.000000,42.000000,1953.000000
25%,86.000000,86.000000,165.000000,58.000000,49.000000,1970.000000
50%,89.000000,89.000000,168.000000,61.000000,52.000000,1983.000000
75%,91.000000,91.000000,173.000000,61.000000,54.000000,1996.000000
max,100.000000,104.000000,190.000000,100.000000,100.000000,2009.000000


1-й способ поиска выбросов. Возьмем признак "рост" и посмотрим, влезаю ли я в интервал допустимых ростов

In [8]:
import math
abs(df_new['L'].mean()-df_new['L'].median())<3*df_new['L'].std()/math.sqrt(df.shape[0])
#условие о симметричности выборки выполняется!!!

True

In [9]:
#тогда можно проверить на выброс так:
abs(df_new['L'].mean()-df_new.iloc[-1]['L'])<3*df_new['L'].std()
#неравенство не выполняется. Следовательно объект - выброс ((((

False

# Метрический способ определения выброса
здесь мы найдем выброс следующим образом.
Вычислим расстояния от каждого объекта до своего ближайшего соседа. Выбросом будет объект с максимальным значением такого рассотяния.

In [10]:
from sklearn import preprocessing
df_num=df_new[['G','T','B','L','W']].values#переводит датафрейм в numpy-массив
min_max_scaler=preprocessing.MinMaxScaler()
scale_df_num=min_max_scaler.fit_transform(df_num)
scale_df_num

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


array([[ 0.43478261,  0.27777778,  0.76923077,  0.375     ,  0.20689655],
       [ 0.2173913 ,  0.27777778,  0.71794872,  0.45      ,  0.20689655],
       [ 0.43478261,  0.27777778,  0.76923077,  0.4       ,  0.17241379],
       ..., 
       [ 0.2173913 ,  0.33333333,  0.71794872,  0.575     ,  0.17241379],
       [ 0.2173913 ,  0.25925926,  0.76923077,  0.5       ,  0.22413793],
       [ 0.82608696,  1.        ,  1.        ,  1.        ,  1.        ]])

In [11]:
from scipy.spatial.distance import pdist
dist_matr=pdist(scale_df_num,'euclidean')# на самом деле это пока не матрица

In [12]:
dist_matr

array([ 0.23561373,  0.04259179,  0.04259179, ...,  0.12812964,
        1.32666186,  1.35071227])

In [17]:
from scipy.spatial.distance import squareform
dist_matr=squareform(dist_matr)# вот так ее нужно сделать матрицей
#dist_matr.shape

In [18]:
inf_array=np.full(dist_matr.shape[0],np.inf)
np.fill_diagonal(dist_matr,inf_array)
dist_to_neigh=dist_matr.min(0)#вектор расстояний до ближайшего соседа
print('max distance is ',dist_to_neigh.max())
print('the number of object with max distance to its neigbour is',np.argmax(dist_to_neigh))
#как видно, объектом с наибольшим рассоянием до его ближайшего соседа оказался я

max distance is  0.907292435763
the number of object with max distance to its neigbour is 604


In [19]:
dist_to_neigh

array([ 0.        ,  0.05      ,  0.        ,  0.        ,  0.        ,
        0.03448276,  0.        ,  0.11545402,  0.01724138,  0.025     ,
        0.08063244,  0.05288918,  0.14388032,  0.16553151,  0.05493926,
        0.03036882,  0.07335695,  0.05172414,  0.05172414,  0.05172414,
        0.        ,  0.0960752 ,  0.06222332,  0.07883162,  0.06538716,
        0.06092142,  0.06859002,  0.05555556,  0.1203979 ,  0.07560601,
        0.05410281,  0.13162023,  0.06538716,  0.17487053,  0.05      ,
        0.07560601,  0.        ,  0.08834724,  0.03036882,  0.01851852,
        0.025     ,  0.06331418,  0.01724138,  0.08012034,  0.        ,
        0.04259179,  0.09061549,  0.05288918,  0.08091433,  0.03089866,
        0.06092142,  0.04468492,  0.18549187,  0.07626042,  0.15651087,
        0.05172414,  0.07626042,  0.1203979 ,  0.05288918,  0.01724138,
        0.09811519,  0.07700802,  0.03448276,  0.01724138,  0.01724138,
        0.11553696,  0.03448276,  0.18251475,  0.07474236,  0.06